In [362]:
import pandas as pd
import regex as re
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

import warnings
warnings.filterwarnings('ignore')

Volem deixar la taula STATUS_TOTAL així:

- cryptopunk_number
- owner
- for_sale: Yes/No
- sale_eth: quantiat ethereum sense el signe/None
- eth_factors: K, M i Y
- sale_dolars: quantiat dolars sense el signe/None
- dolars factors: B
- are_bids: Yes/No
- bids_eth: quantiat ethereum sense el signe/None
- bids_eth_factors: K, M i Y
- bids_dolars: quantiat dolars sense el signe/None
- bids_dolars_factors: B
- bid_placer: usuari que fa la puja/None

In [2]:
df_1 = pd.read_csv('STATUS_TOTAL.csv')

In [3]:
df_1

,Unnamed: 0,owner,for_sale,are_bids,cryptopunk_number
0,0,This punk is currently owned by address 0xe08c32.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,0
1,1,This punk is currently owned by address 0xb88f61.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,1
2,2,This punk is currently owned by address 0x897aea.,This punk has not been listed for sale by its ...,There is a bid of <0.01Ξ ($4) for this punk fr...,2
3,3,This punk is currently owned by address 0xc352b5.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,3
4,4,This punk is currently owned by address 0xc352b5.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,4
...,...,...,...,...,...
9995,9995,This punk is currently owned by address 0x758353.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,9995
9996,9996,This punk is currently owned by address 0xa9bfa9.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,9996
9997,9997,This punk is currently owned by address 0x279679.,This punk is currently for sale by owner for 4...,There are currently no bids on this punk.,9997
9998,9998,This punk is currently owned by address 0xcfd9cc.,This punk has not been listed for sale by its ...,There are currently no bids on this punk.,9998


Eliminem la primera columna, ja que està repetida i canviem l'ordre de les columnes, posant l'id davant:

In [4]:
df_1.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
df_1 = df_1[['cryptopunk_number','owner', 'for_sale', 'are_bids']]

Netejem la columna owner perquè ens quedi només l'identificador de l'usuari:

In [6]:
df_1['owner'].replace({'This punk is currently owned by address':''}, regex=True, inplace=True)

In [7]:
df_1['owner'].replace({'\.':''}, regex=True, inplace=True)

Netegem la columna 'for_sale'. Posem 'No' a tots els registres que no han estat en venta i deixem només el preu dels que han estat venuts:

In [8]:
df_1['for_sale'].value_counts()

This punk has not been listed for sale by its owner.                8079
This punk is currently for sale by owner for 140Ξ ($293,839).        190
This punk is currently for sale by owner for 38.95Ξ ($83,588).       155
This punk is currently for sale by owner for 125Ξ ($274,111).         59
This punk is currently for sale by owner for 34.47Ξ ($73,073).        26
                                                                    ... 
This punk is currently for sale by owner for 28Ξ ($63,110).            1
This punk is currently for sale by owner for 33.98Ξ ($76,589).         1
This punk is currently for sale by owner for 133.70Ξ ($283,334).       1
This punk is currently for sale by owner for 39Ξ ($88,061).            1
This punk is currently for sale by owner for 23.99Ξ ($52,038).         1
Name: for_sale, Length: 1119, dtype: int64

In [9]:
df_1['for_sale'].replace({'This punk has not been listed for sale by its owner.':'No'}, regex=True, inplace=True)

In [10]:
df_1['for_sale'].replace({'This punk is currently for sale by owner for ':''}, regex=True, inplace=True)

In [11]:
df_1['for_sale'].replace({'\.':''}, regex=True, inplace=True)

Separem els preus en dolars i ethereum en dues columnes noves:

In [12]:
df_1[['price_eth','price_dolar']] = df_1['for_sale'].str.split(expand=True)

In [13]:
df_1['for_sale'].replace({'[^"No"]+':'Yes'}, regex=True, inplace=True)

In [14]:
df_1['for_sale'].value_counts()

No     8079
Yes    1921
Name: for_sale, dtype: int64

In [15]:
df_1['for_sale'].value_counts()

No     8079
Yes    1921
Name: for_sale, dtype: int64

Netegem els registres de Bids:

In [16]:
df_1['are_bids'].replace({'There are currently no bids on this punk.':'No'}, regex=True, inplace=True)

In [17]:
df_1['are_bids'].replace({'There is a bid of ':''}, regex=True, inplace=True)

In [18]:
df_1['are_bids']

0                                              No
1                                              No
2        <0.01Ξ ($4) for this punk from 0x7f6498.
3                                              No
4                                              No
                          ...                    
9995                                           No
9996                                           No
9997                                           No
9998                                           No
9999    0.22Ξ ($489) for this punk from 0x20ed17.
Name: are_bids, Length: 10000, dtype: object

Creem les columnes necessaries per incloure tota la informació de bids:

In [19]:
df_1['bids_price'] = df_1['are_bids']

In [20]:
df_1['bids_placer'] = df_1['are_bids']

In [21]:
df_1.loc[df_1['are_bids']!='No', 'are_bids'] = df_1.loc[df_1['are_bids']!='No']['are_bids'].replace({'\w':'', '\W':''}, regex=True)

In [22]:
df_1['are_bids'].replace({'':'Yes'}, regex=True, inplace=True)

In [23]:
df_1['are_bids'].value_counts()

No     9108
Yes     892
Name: are_bids, dtype: int64

Netegem 'price_eth': FALTA SABER QUÈ FER AMB ELS FACTORIALS 'K', 'M' i 'Y' !

In [24]:
df_1['price_eth'].replace({'Ξ':''}, regex=True, inplace=True)

In [25]:
df_1['price_eth'].replace({'No':'None'}, regex=True, inplace=True)

Netegem 'price_dolar':

In [26]:
df_1['price_dolar'].replace({'\(':'', '\)':'', '\$':'', ',':''}, regex=True, inplace=True)

In [27]:
df_1.price_dolar.value_counts()

293839    190
83588     155
274111     59
73073      26
49918      23
         ... 
37319       1
442676      1
201322      1
119303      1
70798       1
Name: price_dolar, Length: 1107, dtype: int64

In [28]:
df_1['dolar_factors'] = df_1['price_dolar'].str.extract(r'([a-zA-Z])')

In [29]:
df_1['eth_factors'] = df_1['price_eth'].str.extract(r'([a-zA-Z])')

In [30]:
df_1['eth_factors'].value_counts()

N    8079
K      44
Y      20
M       1
Name: eth_factors, dtype: int64

In [31]:
df_1['dolar_factors'].value_counts()

M    114
Y     20
B      1
Name: dolar_factors, dtype: int64

Separem la columna de de bids en si hi ha bid o no, l'usuari que l'ha fet, el preu en dolars i el preu en ethereum:

In [32]:
df_1['bids_price'].replace({'<':''}, regex=True, inplace=True)

In [187]:
df_1['bids_dolars'] = df_1['bids_price'].str.extract(r'(\$\d+(\,\d+)?[a-zA-Z]?)')

In [51]:
df_1['bids_dolars'].replace({'$':''}, regex=True, inplace=True)

In [252]:
df_1['bids_eth'] = df_1['bids_price'].str[:6]

KeyError: 'bids_price'

In [ ]:
df_1['bids_eth'].head(200)

In [223]:
df_1['bids_eth'].replace({'Ξ':'', 'No':'', '\(\$\d?': ''}, regex=True, inplace=True)

In [229]:
df_1['bids_placer'] = df_1['bids_price'].str.extract(r'(from\s\w+)')

In [230]:
df_1['bids_placer'].replace({'from':''}, regex=True, inplace=True)

In [232]:
df_1.drop('bids_price', axis=1, inplace=True)

In [233]:
df_1.columns

Index(['cryptopunk_number', 'owner', 'for_sale', 'are_bids', 'price_eth', 'price_dolar', 'bids_placer', 'dolar_factors', 'eth_factors', 'bids_dolars', 'bids_eth'], dtype='object')

In [234]:
df_1 = df_1[['cryptopunk_number', 'owner', 'for_sale', 'price_dolar', 'dolar_factors', 'price_eth', 'eth_factors', 
             'are_bids', 'bids_placer', 'bids_dolars', 'bids_eth']]

In [235]:
df_1

,cryptopunk_number,owner,for_sale,price_dolar,dolar_factors,price_eth,eth_factors,are_bids,bids_placer,bids_dolars,bids_eth
0,0,0xe08c32,No,None,NaN,None,N,No,NaN,NaN,
1,1,0xb88f61,No,None,NaN,None,N,No,NaN,NaN,
2,2,0x897aea,No,None,NaN,None,N,Yes,0x7f6498,$4,0.01
3,3,0xc352b5,No,None,NaN,None,N,No,NaN,NaN,
4,4,0xc352b5,No,None,NaN,None,N,No,NaN,NaN,
...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0x758353,No,None,NaN,None,N,No,NaN,NaN,
9996,9996,0xa9bfa9,No,None,NaN,None,N,No,NaN,NaN,
9997,9997,0x279679,Yes,991742,NaN,450,NaN,No,NaN,NaN,
9998,9998,0xcfd9cc,No,None,NaN,None,N,No,NaN,NaN,


### TXN_TOTAL

Volem deixar la taula TXN_TOTAL així:

- cryptopunk_number
- txn_type
- from
- to
- amount_dolars
- dolars_factors:
- amount_ethereum
- ethereum_factors:
- txn_date


In [88]:
df_2 = pd.read_csv('TXN_TOTAL.csv')

In [89]:
df_2

,Unnamed: 0,Unnamed: 0.1,txn_type,from,to,amount,txn_date,cryptopunk_number
0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,1,1,Bid Withdrawn,0x983ace,NaN,250Ξ ($1.03M),"May 10, 2021",0.0
2,2,2,Bid,0x983ace,NaN,"250Ξ ($537,615)","Apr 11, 2021",0.0
3,3,3,Bid Withdrawn,0xd7510a,NaN,"100Ξ ($160,973)","Mar 03, 2021",0.0
4,4,4,Bid,0xd7510a,NaN,"100Ξ ($188,897)","Feb 20, 2021",0.0
...,...,...,...,...,...,...,...,...
133269,133269,70605,Bid,0xa0a59c,NaN,0.25Ξ ($69),"Jul 03, 2017",9999.0
133270,133270,70606,Bid,0x717403,NaN,0.20Ξ ($51),"Jun 26, 2017",9999.0
133271,133271,70607,Bid,ddaavvee,NaN,0.20Ξ ($65),"Jun 23, 2017",9999.0
133272,133272,70608,Bid,0x5b098b,NaN,0.15Ξ ($50),"Jun 23, 2017",9999.0


In [90]:
df_2.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [118]:
df_2.drop(0, inplace=True)

In [119]:
df_2.txn_type.value_counts().head(10)

Offered            43487
Bid                21934
Sold               13848
Bid Withdrawn      12429
Transfer           10687
Claimed            10000
Offer Withdrawn     6534
(Wrap)              2297
(Unwrap)            2058
7068                   1
Name: txn_type, dtype: int64

In [120]:
df_2['from'].value_counts(normalize=True)

Pranksy      0.036220
WrappedC…    0.032536
0x717403     0.032520
Hemba        0.019430
0x7760e0     0.017090
               ...   
0xa2af28…    0.000016
0xece772     0.000016
0x60385b     0.000016
0x143418     0.000016
0x15547b     0.000016
Name: from, Length: 4052, dtype: float64

In [121]:
df_2['to'].value_counts(normalize=True)

WrappedC…    0.058542
0xc352b5     0.027219
Hemba        0.024874
0x00d7c9     0.023549
Pranksy      0.018834
               ...   
0x3db26e     0.000025
0xcc72f7     0.000025
0x3d19c5     0.000025
0xbdeef4     0.000025
0x59fe45     0.000025
Name: to, Length: 4343, dtype: float64

In [122]:
df_2['amount'].value_counts()

0.05Ξ ($14)         1025
0.05Ξ ($13)          800
<0.01Ξ ($<0.01)      657
0.05Ξ ($16)          569
0.03Ξ ($10)          387
                    ... 
52Ξ ($116,474)         1
125Ξ ($262,280)        1
24.49Ξ ($86,405)       1
41.50Ξ ($76,807)       1
31Ξ ($106,010)         1
Name: amount, Length: 52622, dtype: int64

In [124]:
df_2['txn_date'].value_counts()

Jun 23, 2017    10452
May 03, 2021     1635
Mar 28, 2021     1287
Sep 28, 2020     1193
Feb 21, 2021     1117
                ...  
Sep 07, 2019        1
Apr 03, 2019        1
Nov 08, 2017        1
Jan 30, 2018        1
Mar 28, 2020        1
Name: txn_date, Length: 1367, dtype: int64

In [125]:
df_2['cryptopunk_number'].value_counts()

3914.0    111
6704.0    104
1737.0    102
7421.0    102
8951.0     99
         ... 
8496.0      1
2171.0      1
2797.0      1
4981.0      1
5459.0      1
Name: cryptopunk_number, Length: 10000, dtype: int64

In [126]:
df_2.columns

Index(['txn_type', 'from', 'to', 'amount', 'txn_date', 'cryptopunk_number'], dtype='object')

In [127]:
df_2 = df_2[['cryptopunk_number','txn_type', 'from', 'to', 'amount', 'txn_date']]

In [135]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133273 entries, 1 to 133273
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   cryptopunk_number  123274 non-null  float64       
 1   txn_type           133273 non-null  object        
 2   from               63253 non-null   object        
 3   to                 39237 non-null   object        
 4   amount             91698 non-null   object        
 5   txn_date           123274 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 7.1+ MB


Hi ha un nul per cada cryptopunk:

In [132]:
df_2['cryptopunk_number'].isna().sum()

9999

In [134]:
df_2["txn_date"]= pd.to_datetime(df_2["txn_date"])

In [136]:
df_2['txn_date'].value_counts()

2017-06-23    10452
2021-05-03     1635
2021-03-28     1287
2020-09-28     1193
2021-02-21     1117
              ...  
2017-10-22        1
2019-11-17        1
2020-07-18        1
2018-08-14        1
2018-07-17        1
Name: txn_date, Length: 1367, dtype: int64

In [137]:
df_2.isna().sum()

cryptopunk_number     9999
txn_type                 0
from                 70020
to                   94036
amount               41575
txn_date              9999
dtype: int64

Veiem que al final de cada crypto punk hi ha una fila de barrera on els valors son nulls. Les busquem totes i les eliminem:

In [156]:
i = df_2['cryptopunk_number'].isna()

In [161]:
i = df_2[i].index

In [163]:
df_2.drop(i, inplace=True)

In [164]:
df_2.isna().sum()

cryptopunk_number        0
txn_type                 0
from                 60021
to                   84037
amount               31576
txn_date                 0
dtype: int64

Passem la columna de id dels punks a INT:

In [ ]:
df_2['cryptopunk_number'] = df_2['cryptopunk_number'].astype(int)

Comencem amb els regex:

In [165]:
df_2['amount'].replace({'<':''}, regex=True, inplace=True)

Creem les columnes per dolars i Ethereum, omplim amb NaN els espais buits de la columna 'amount':

In [385]:
df_2['amount'].replace(r'^(?!\d).*$', np.nan, regex=True, inplace=True)

Columna dolars, agafant des del tercer caràcter de l'string i eliminant allò que ens sobra:

In [355]:
df_2['amount_dolars'] = df_2['amount'].str[3:]

In [357]:
df_2['amount_dolars'].replace({'\.?\d*[a-zA-Z]?Ξ':'','\s':'', '\$':'', '\(':'',  '\)':'', '\,':'', '[a-zA-Z]':''}, regex=True, inplace=True)

Columna d'ethereum, agafant des del setè caràcter de l'string i eliminant allò que ens sobra:

In [391]:
df_2['amount_eth'] = df_2['amount'].str[:7]

In [392]:
df_2['amount_eth'].replace({'Ξ':'','\s':'', '\(':'','\)':'','\$':'','\$\d+?': '', '\,':'', '[a-zA-Z]':''}, regex=True, inplace=True)

Convertim les columnes a numèric, per fer-ho hem d'omplir els empty strings amb NaN:

In [365]:
df_2.replace(r'', np.nan, regex=True, inplace=True)

In [366]:
df_2['amount_dolars'] = df_2['amount_dolars'].astype(float)

In [393]:
df_2['amount_eth'].replace(r'^(?!\d).*$', np.nan, regex=True, inplace=True)

In [394]:
df_2['amount_eth'] = df_2['amount_eth'].astype(float)

Fem les columnes de factors corresponents:

In [345]:
df_2['dolar_factors'] = df_2['amount_dolars'].str.extract(r'([a-zA-Z])')

In [346]:
df_2['dolar_factors'].value_counts()

M    473
Y     28
T      3
B      3
Z      2
P      1
Name: dolar_factors, dtype: int64

In [322]:
df_2['eth_factors'] = df_2['amount_eth'].str.extract(r'([a-zA-Z])')

In [325]:
df_2['eth_factors'].value_counts()

K    235
Z      3
M      2
B      2
T      1
Y      1
Name: eth_factors, dtype: int64

In [399]:
df_2

,cryptopunk_number,txn_type,from,to,amount,txn_date,amount_dolars,amount_eth,dolar_factors,eth_factors
1,0,Bid Withdrawn,0x983ace,NaN,250Ξ ($1.03M),2021-05-10,1.03,250.00,1000000.0,NaN
2,0,Bid,0x983ace,NaN,"250Ξ ($537,615)",2021-04-11,537615.00,250.00,NaN,NaN
3,0,Bid Withdrawn,0xd7510a,NaN,"100Ξ ($160,973)",2021-03-03,160973.00,100.00,NaN,NaN
4,0,Bid,0xd7510a,NaN,"100Ξ ($188,897)",2021-02-20,188897.00,100.00,NaN,NaN
5,0,Bid Withdrawn,natealex,NaN,"69Ξ ($25,532)",2020-09-16,25532.00,692.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
133269,9999,Bid,0xa0a59c,NaN,0.25Ξ ($69),2017-07-03,69.00,0.25,NaN,NaN
133270,9999,Bid,0x717403,NaN,0.20Ξ ($51),2017-06-26,51.00,0.20,NaN,NaN
133271,9999,Bid,ddaavvee,NaN,0.20Ξ ($65),2017-06-23,65.00,0.20,NaN,NaN
133272,9999,Bid,0x5b098b,NaN,0.15Ξ ($50),2017-06-23,50.00,0.15,NaN,NaN


In [398]:
factors = {'K': 10**3,
          'M': 10**6,
          'G': 10**9,
          'T': 10**12,
          'P': 10**15,
          'E': 10**18,
          'Z': 10**21,
          'Y': 10**24,
          'B': 10**9,
          }

df_2['eth_factors'] = df_2['eth_factors'].map(factors).astype(float)
df_2['dolar_factors'] = df_2['dolar_factors'].map(factors).astype(float)

Revisar tema factors, els mès grans fent value counts, per mirar que no doni uns valors massa grans. Fer també per la TAULA 1.

In [ ]:
# df_2.drop('amount', inplace=True)